<a href="https://colab.research.google.com/github/Hou-jing/-repository-experiment/blob/main/bert_model_fune_2(%E5%90%ABmodel.summary).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
train_sentence=['The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.', 'The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord.']

In [2]:
train_relation=['Component-Whole', 'Other']

In [3]:
from tensorflow.keras.preprocessing import text
#facts, accu_label, article_label, imprison_label=load_data()
somestr = train_sentence
tok=text.Tokenizer() #初始化标注器
token=tok.fit_on_texts(somestr) #学习出文本的字典
word_index = tok.word_index#查看对应的单词和数字的映射关系dict
print(word_index)
word_docs=tok.word_docs
print(word_docs)#document_count, word_counts, word_docs, index_docs, index_word, word_index

sequences = tok.texts_to_sequences(somestr) #通过texts_to_sequences 这个dict可以将每个string的每个词转成数字
print(sequences)

{'e1': 1, 'e2': 2, 'the': 3, 'of': 4, 'system': 5, 'as': 6, 'described': 7, 'above': 8, 'has': 9, 'its': 10, 'greatest': 11, 'application': 12, 'in': 13, 'an': 14, 'arrayed': 15, 'configuration': 16, 'antenna': 17, 'elements': 18, 'child': 19, 'was': 20, 'carefully': 21, 'wrapped': 22, 'and': 23, 'bound': 24, 'into': 25, 'cradle': 26, 'by': 27, 'means': 28, 'a': 29, 'cord': 30}
defaultdict(<class 'int'>, {'elements': 1, 'described': 1, 'its': 1, 'arrayed': 1, 'above': 1, 'configuration': 1, 'system': 1, 'application': 1, 'the': 2, 'greatest': 1, 'e1': 2, 'of': 2, 'e2': 2, 'antenna': 1, 'an': 1, 'in': 1, 'has': 1, 'as': 1, 'child': 1, 'carefully': 1, 'was': 1, 'bound': 1, 'into': 1, 'cradle': 1, 'a': 1, 'cord': 1, 'by': 1, 'means': 1, 'wrapped': 1, 'and': 1})
[[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 16, 1, 4, 17, 2, 18, 2], [3, 1, 19, 1, 20, 21, 22, 23, 24, 25, 3, 2, 26, 2, 27, 28, 4, 29, 30]]


In [4]:
!pip install transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


     |████████████████████████████████| 2.9 MB 15.3 MB/s 
     |████████████████████████████████| 636 kB 62.5 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 3.3 MB 61.3 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
def parse_sentence(sentence):
  bert_tks=[]

  for sentence in sentence:
    sentence=sentence.replace('<e1>', '<e1>').replace('</e1>', '<e1>').replace('<e2>', '<e1>').replace('</e2>', '<e1>')
    trunks=sentence.split('<e1>')
    bert_tokens=[]
    bert_tokens.append('[CLS]')
    if len(trunks)!=5:
      print('error')
    else:
      for i,trunk in enumerate(trunks):
        tks=tokenizer.tokenize(trunk)
        # tks=tok.fit_on_texts(trunk)
        bert_tokens.extend(tks)
        if i==0 or i==1:
          bert_tokens.append('[EN1]')
        elif i == 2 or i ==3:
                bert_tokens.append('[EN2]')

    bert_tokens.append("[SEP]")

    bert_tks.append(bert_tokens)
  return bert_tks
bert_token=parse_sentence(train_sentence)
print(bert_token)

[['[CLS]', 'the', 'system', 'as', 'described', 'above', 'has', 'its', 'greatest', 'application', 'in', 'an', 'array', '##ed', '[EN1]', 'configuration', '[EN1]', 'of', 'antenna', '[EN2]', 'elements', '[EN2]', '.', '[SEP]'], ['[CLS]', 'the', '[EN1]', 'child', '[EN1]', 'was', 'carefully', 'wrapped', 'and', 'bound', 'into', 'the', '[EN2]', 'cradle', '[EN2]', 'by', 'means', 'of', 'a', 'cord', '.', '[SEP]']]


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq=[tokenizer.convert_tokens_to_ids(toke) for toke in bert_token]
print(seq)

[[101, 1996, 2291, 2004, 2649, 2682, 2038, 2049, 4602, 4646, 1999, 2019, 9140, 2098, 100, 9563, 100, 1997, 13438, 100, 3787, 100, 1012, 102], [101, 1996, 100, 2775, 100, 2001, 5362, 5058, 1998, 5391, 2046, 1996, 100, 18293, 100, 2011, 2965, 1997, 1037, 11601, 1012, 102]]


In [7]:
train_ids=pad_sequences(sequences=seq,maxlen=128,dtype='long',padding='post')
print(train_ids[0])

[  101  1996  2291  2004  2649  2682  2038  2049  4602  4646  1999  2019
  9140  2098   100  9563   100  1997 13438   100  3787   100  1012   102
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [8]:
import numpy as np
def build_entitymask(bert_tok):
  mask=[[i for i,x in enumerate(msk) if x=='[EN1]' or x=='[EN2]'] for msk in bert_tok]
  print(mask)
  '''mask=[]
  for mak in bert_tok:
    for x in range(len(mak)):
      if mak[x]=='[EN1]' of msk[x]=='[EN2]':
        mask.append(x)
        print(mask)'''
  e1_masks=[]
  e2_masks=[]
  for i,mark in enumerate(mask):
    e1_mask=np.zeros((128,))
    e2_mask=np.zeros((128,))
    print(mask[i][0]+1)
    print(mask[i][2]+1)
    e1_mask[mask[i][0]+1]=1
    e2_mask[mask[i][2]+1]=1
    e1_masks.append(e1_mask)
    e2_masks.append(e2_mask)
  return e1_masks,e2_masks
train_e1_mask,train_e2_mask=build_entitymask(bert_token)
print('训练数据第一个e1的mask{}'.format(train_e1_mask[0]))
print('测试数据第一个e2的mask{}'.format(train_e2_mask[0]))

[[14, 16, 19, 21], [2, 4, 12, 14]]
15
20
3
13
训练数据第一个e1的mask[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
测试数据第一个e2的mask[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
from sklearn.preprocessing.label import LabelEncoder
label=LabelEncoder()
label_rel=label.fit_transform(train_relation)#fit转为集合，transform转为编码
print(label_rel)
#查看编码对应的原始值
rel=label.inverse_transform([0,1])
print(rel)

[0 1]
['Component-Whole' 'Other']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


In [10]:
#注意力mask
train_attention_mask=[[float(i>0) for i in ii] for ii in train_ids]
print (train_attention_mask[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
!pip install tensorflow_text
import tensorflow_text as text
import tensorflow_hub as hub
BERT_MODEL_HUB ="https://hub.tensorflow.google.cn/tensorflow/bert_zh_L-12_H-768_A-12/2"

In [ ]:
from tensorflow.keras import backend as K
bert_model=hub.load(BERT_MODEL_HUB)
# bert_model.signatures['tokens']
train_segment_ids=np.zeros_like(train_ids)



In [ ]:
inputs=train_ids, train_attention_mask, train_segment_ids
train_ids, train_attention_mask, train_segment_ids= [K.cast(x, dtype="int32") for x in inputs]
module = hub.KerasLayer(BERT_MODEL_HUB,trainable=False)
pooled_output, sequence_output = module([train_attention_mask,train_segment_ids,train_ids])
print(pooled_output[1],sequence_output[1])
# bert_inputs = dict(
#             input_ids=train_ids, input_mask=train_attention_mask, segment_ids=train_segment_ids
#         )
        
#         # Use "pooled_output" for classification tasks on an entire sentence.
#         # Use "sequence_outputs" for token-level output.
# result = bert_model(inputs=bert_inputs,as_dict=True)

In [67]:
bert_input=sequence_output
e1_mask=tf.keras.layers.Input(shape=(128,))
e2_mask=tf.keras.layers.Input(shape=(128,))
import tensorflow.keras.backend as K

In [55]:
h0=sequence_output[:,0,:]
print(h0)

tf.Tensor(
[[-0.04353601  0.9273969   0.48475364 ...  0.8306807  -0.14067191
   0.08023699]
 [-0.0698668   0.927066    0.4697714  ...  0.8354876  -0.17368418
   0.09390133]], shape=(2, 768), dtype=float32)


In [59]:
e1_mask=tf.keras.layers.Input(shape=(128,))
h1 = K.sum(sequence_output * K.expand_dims(e1_mask, -1), axis = 1, keepdims=False)
print(h1)

KerasTensor(type_spec=TensorSpec(shape=(2, 768), dtype=tf.float32, name=None), name='tf.math.reduce_sum_1/Sum:0', description="created by layer 'tf.math.reduce_sum_1'")


In [60]:
e1_lenth=K.sum(e1_mask,axis=1,keepdims=True)

In [65]:
print(e1_mask.shape)
print(K.expand_dims(e1_mask,-1).shape)
print(sequence_output.shape)

(None, 128)
(None, 128, 1)
(2, 128, 768)


In [77]:
h1 = h1 / (e1_lenth + 1e-8)
h1

<KerasTensor: shape=(2, 768) dtype=float32 (created by layer 'tf.math.truediv_7')>

In [81]:
h1 = K.tanh(h1)
h1 = tf.keras.layers.Dropout(0.1)(h1)
h1=tf.keras.layers.Dense(128)(h1)
h1

<KerasTensor: shape=(2, 128) dtype=float32 (created by layer 'dense_3')>

In [82]:
bert_input=sequence_output
e1_mask=tf.keras.layers.Input(shape=(128,))
e2_mask=tf.keras.layers.Input(shape=(128,))
import tensorflow.keras.backend as K

In [84]:
      def build():
        h0=K.tanh(bert_input[:,0,:])
        h1=K.sum(bert_input*K.expand_dims(e1_mask,-1),axis=1,keepdims=False)
        h2 = K.sum(bert_input * K.expand_dims(e2_mask, -1), axis=1, keepdims=False)
        e1_len=K.sum(e1_mask,axis=1,keepdims=True)
        e2_len = K.sum(e2_mask, axis=1, keepdims=True)
        h1 = h1 / (e1_len + 1e-8)
        h2 = h2 / (e2_len + 1e-8)

        h1 = K.tanh(h1)
        h2 = K.tanh(h2)

        # add dropout
        h0 = tf.keras.layers.Dropout(0.1)(h0)
        h1 = tf.keras.layers.Dropout(0.1)(h1)
        h2 = tf.keras.layers.Dropout(0.1)(h2)

        #全连接层
        h0=tf.keras.layers.Dense(128)(h0)
        h1=tf.keras.layers.Dense(128)(h1)
        h2=tf.keras.layers.Dense(128)(h2)
        return K.concatenate([h0,h1,h2],-1) 
      average_output=build() 
      print(average_output)  

KerasTensor(type_spec=TensorSpec(shape=(2, 384), dtype=tf.float32, name=None), name='tf.concat_1/concat:0', description="created by layer 'tf.concat_1'")


In [98]:
x=tf.keras.layers.Dropout(0.1)(average_output)
out=tf.keras.layers.Dense(2)(x)



In [99]:
print(out)

KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='dense_24/BiasAdd:0', description="created by layer 'dense_24'")


In [97]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow.keras.backend as K
import tensorflow_hub as hub
BERT_MODEL_HUB ="https://hub.tensorflow.google.cn/tensorflow/bert_zh_L-12_H-768_A-12/2"
from tensorflow.keras.layers import Input
input_mask=Input(shape=(128,),name='input_mask')
segment_ids=Input(shape=(128,),name='segment_ids')
inputs_ids=Input(shape=(128,),name='input_ids')
call_input=[input_mask,segment_ids,inputs_ids]
e1_mask=Input(shape=(128,),name='e1_mask')
e2_mask=Input(shape=(128,),name='e2_mask')
def call(inputs):
    module = hub.KerasLayer(BERT_MODEL_HUB, trainable=False)
    input_ids, input_mask, segment_ids =[K.cast(x, dtype="int32") for x in inputs]
    pooled_output, sequence_output=module([input_mask,segment_ids,input_ids])
    return sequence_output
bert_input=call(call_input)


def build():
    h0 = K.tanh(bert_input[:, 0, :])
    h1 = K.sum(bert_input * K.expand_dims(e1_mask, -1), axis=1, keepdims=False)
    h2 = K.sum(bert_input * K.expand_dims(e2_mask, -1), axis=1, keepdims=False)
    e1_len = K.sum(e1_mask, axis=1, keepdims=True)
    e2_len = K.sum(e2_mask, axis=1, keepdims=True)
    h1 = h1 / (e1_len + 1e-8)
    h2 = h2 / (e2_len + 1e-8)

    h1 = K.tanh(h1)
    h2 = K.tanh(h2)

    # add dropout
    h0 = tf.keras.layers.Dropout(0.1)(h0)
    h1 = tf.keras.layers.Dropout(0.1)(h1)
    h2 = tf.keras.layers.Dropout(0.1)(h2)

    # 全连接层
    h0 = tf.keras.layers.Dense(128)(h0)
    h1 = tf.keras.layers.Dense(128)(h1)
    h2 = tf.keras.layers.Dense(128)(h2)
    return K.concatenate([h0, h1, h2], -1)


average_output = build()
print(average_output)
x=tf.keras.layers.Dropout(0.1)(average_output)
out=tf.keras.layers.Dense(2,activation='softmax')(x)

input=[input_mask,segment_ids,inputs_ids,e1_mask,e2_mask]
model=tf.keras.models.Model(input,out)
model.compile(tf.keras.optimizers.Adam(lr=2e-5),loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
model.summary()



KerasTensor(type_spec=TensorSpec(shape=(None, 384), dtype=tf.float32, name=None), name='tf.concat_4/concat:0', description="created by layer 'tf.concat_4'")
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf.cast_13 (TFOpLambda)         (N

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# 预训练bert模型地址，其中，末尾的 2 为该模型的版本号
# 目前还有很多模型是基于 TF1.0 的，选择的过程中请注意甄别，有些模型会明确写出来是试用哪个版本，或者，检查使用是否是 tfhub 0.5.0 或以上版本的 API hub.load(url) ，在之前版本使用的hub.Module(url)
 
BERT_URL = "https://hub.tensorflow.google.cn/tensorflow/bert_zh_L-12_H-768_A-12/2"
 
max_seq_length = 256  # 定义序列长度
 
# 定义3个输入
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,),
                                      dtype=tf.int32,name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,),
                                   dtype=tf.int32,name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,),
                                    dtype=tf.int32,name="segment_ids")
# 构建预训练模型+自定义层
module = hub.KerasLayer(BERT_URL,trainable=True)
pooled_output, sequence_output = module([input_mask,segment_ids,input_word_ids])
out = tf.keras.layers.Dense(2,activation="softmax")(pooled_output)
 
model = tf.keras.Model(inputs=[input_word_ids,input_mask,segment_ids],outputs=out)
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])
 
# 这里确保输入和输出是numpy.array类型
model.fit([inputIds, inputMask, segmentIds],labels,
          batch_size=1,epochs=20,
          validation_data=([inputIds, inputMask, segmentIds],labels),
          verbose=1)

In [81]:

print(train_ids.shape)
print(type(train_ids))
print(type(train_segment_ids),type(np.array(train_attention_mask)))
from tensorflow.keras import backend as K
train_segment_ids=K.cast(train_segment_ids,tf.int32)

# print(train_segment_ids)
# train_ids=tf.keras.layers.Input(shpe=(128,),dtype=tf.int32,name='train_ids')
# train_attention_mask=tf.keras.layers.Input(shape=(128,),dtype=tf.int32,name='train_attention_mask')
# train_segment_ids=tf.keras.layers.Input(shape=(128,),dtype=tf.int32,name='train_segment_ids')
bert_input=dict(train_ids=train_ids,train_attention_mask=train_attention_mask,train_segment_ids=train_segment_ids)
bert_result=bert_model(bert_input)

(2, 128)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


ValueError: ignored

In [65]:
from tensorflow.keras import backend as K
import tensorflow_hub as hub
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
class BertLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(BertLayer, self).__init__()
        self.output_size=768
        self.trainable=True
        self.bert=hub.load(BERT_MODEL_HUB)
    # def bulid(self,inputs):
    #     self.bert=hub.load(BERT_MODEL_HUB)
    #     super(BertLayer, self).bulid(inputs)
        # self.bert=hub.Module(BERT_MODEL_HUB,self.trainable,name='{}_module'.format(self.name))
    def call(self,inputs):
        self.bert=hub.load(BERT_MODEL_HUB)
        self.bert.signatures['default'](['my text', 'batch'])
        # self.bert=hub.Module(BERT_MODEL_HUB,self.trainable,name='{}_module'.format(self.name))
        inputs=[K.cast(x,dtype='int32') for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs=dict(inputs_ids=input_ids,inputs_mask=input_mask,segment_ids=segment_ids)
        result=self.bert(bert_inputs,signature='tokens',as_dict=True)
        return result['sequence_output']
bert_model=BertLayer()
inputs=[train_ids,train_attention_mask,np.zeros_like(train_ids)]
bert_model(inputs)

KeyError: ignored

SyntaxError: ignored

In [2]:
import tensorflow as tf
from tensorflow.keras import backend as  K
import tensorflow_hub as hub
# A dependency of the preprocessing for BERT inputs
!pip install tensorflow_text
import tensorflow_text as text

In [18]:

bert_model_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1'
# bert_preprocess_model = hub.load(bert_model_preprocess)
bert_preprocess_model=hub.KerasLayer(bert_model_preprocess)

text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)
print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')


Keys       : ['input_word_ids', 'input_type_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
#调用方法1
bert_model="https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/mnli/2"
bert_model=hub.KerasLayer(bert_model)
bert_model(text_preprocessed)

In [17]:
#2
BERT_MODEL = "https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/2" # @param {type: "string"} ["https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/2", "https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/mnli/2", "https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/qnli/2", "https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/qqp/2", "https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/squad2/2", "https://hub.tensorflow.google.cn/google/experts/bert/wiki_books/sst2/2",  "https://hub.tensorflow.google.cn/google/experts/bert/pubmed/2", "https://tfhub.dev/google/experts/bert/pubmed/squad2/2"]
# Preprocessing must match the model, but all the above use the same.

# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
bert_model=hub.load(BERT_MODEL)
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.06959195 -0.24941234 -0.52000475  0.04785056  0.46641964  0.3956707
  0.72048706  0.24604926  0.017136    0.38573763 -0.47015676  0.47310936]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[-0.06970463 -0.2547861  -0.5763463  ... -0.7816097  -2.331598
   0.63859   ]
 [-0.14387189 -0.31786576 -0.40908676 ... -0.7608944  -2.4582336
   0.818131  ]
 [-0.06832582 -0.2585022  -0.4815255  ... -0.70945704 -2.2907138
   0.85197234]
 ...
 [-0.0240149  -0.3927874  -0.5207361  ... -0.6500671  -2.2678778
   0.76076955]
 [-0.0245671  -0.38878685 -0.51834476 ... -0.65286624 -2.2672896
   0.76042837]
 [-0.02994342 -0.38545093 -0.51554286 ... -0.6512956  -2.2640586
   0.75882584]]


In [ ]:
!pip install bert-tensorflow
!pip install tensorflow-hub
from bert import tokenization

                                                

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [ ]:
import tensorflow_hub as hub
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
with tf.Graph().as_default():
  bert_module = hub.Module(BERT_MODEL_HUB)
  tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
  vocab_file, do_lower_case =tokenization_info["vocab_file"],tokenization_info["do_lower_case"]
Fulltoken=tokenization.FullTokenizer(vocab_file=vocab_file,do_lower_case=True)
token=Fulltoken.tokenize(train_sentence)

In [ ]:
import gensim
text='The system as described configuration'
token=gensim.utils.tokenize(text,lower=True)
print(token)

<generator object simple_tokenize at 0x7f909916f9d0>


In [ ]:
from tensorflow.keras.preprocessing import text
#facts, accu_label, article_label, imprison_label=load_data()
somestr = ['ha ha gua angry','howa ha gua excited naive']
tok=text.Tokenizer() #初始化标注器
tok.fit_on_texts(somestr) #学习出文本的字典
word_index = tok.word_index#查看对应的单词和数字的映射关系dict
print(word_index)
sequences = tok.texts_to_sequences(somestr) #通过texts_to_sequences 这个dict可以将每个string的每个词转成数字
print(sequences)


{'ha': 1, 'gua': 2, 'angry': 3, 'howa': 4, 'excited': 5, 'naive': 6}
[[1, 1, 2, 3], [4, 1, 2, 5, 6]]
